## Project will do the following:
## Based on the name of an existing kickstarter project, 5 of the most similar projects will be
## recommended. Afterwards, each of these 5 projects will have a success/failure prediction

In [40]:
import pandas as pd
import json
from rake_nltk import Rake
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.max_colwidth', -1)

In [41]:
#dftest = pd.read_csv('ks-projects-201801.csv')

dfnew = pd.read_csv('https://raw.githubusercontent.com/L-Lewis/Kickstarter-success-machine-learning/master/data/Kickstarter000.csv')

In [42]:
def combine_features(row):
    category = json.loads(row['category'])
    r = Rake()
    r.extract_keywords_from_text(row['blurb'])
    arr = r.get_ranked_phrases()
    return category['name']+" "+category['slug']+" "+ " ".join(arr)

dfnew["combinedfeatures"] = dfnew.apply(combine_features, axis=1)

In [43]:
def get_title_from_index(index):
    return dfnew[dfnew.index == index]['name']

def get_index_from_title(title):
    return dfnew[dfnew['name']==title].index[0]

In [71]:

cv = CountVectorizer()
count_matrix = cv.fit_transform(dfnew["combinedfeatures"])
cosine_sim = cosine_similarity(count_matrix)

userlikes = "New Final Round Album"
index = get_index_from_title(userlikes)
similar_projects = list(enumerate(cosine_sim[index]))
sortedprojects = sorted(similar_projects, key=lambda x: x[1], reverse=True)

dftest = pd.DataFrame()

for i in range(1,6):
    index = sortedprojects[i][0]
    print(sortedprojects[i][1])
    dftest=dftest.append(dfnew[dfnew.index == index][['category', 'name', 'state', 'urls', 'photo']])
    
dftest

0.50709255283711
0.4879500364742666
0.4767312946227961
0.44999999999999996
0.4472135954999579


,category,name,state,urls,photo
1051,"{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""position"":17,""parent_id"":14,""color"":10878931,""urls"":{""web"":{""discover"":""http://www.kickstarter.com/discover/categories/music/rock""}}}",Moon Debris LP,successful,"{""web"":{""project"":""https://www.kickstarter.com/projects/1557938215/moon-debris-lp?ref=discovery_category_newest"",""rewards"":""https://www.kickstarter.com/projects/1557938215/moon-debris-lp/rewards""}}","{""key"":""assets/011/398/387/4488ea1b7a56306c87cbffcfeedd9ed3_original.jpg"",""full"":""https://ksr-ugc.imgix.net/assets/011/398/387/4488ea1b7a56306c87cbffcfeedd9ed3_original.jpg?ixlib=rb-1.1.0&crop=faces&w=560&h=315&fit=crop&v=1463682113&auto=format&frame=1&q=92&s=0b5b97a5b650d3aeb20e3cfd1504d343"",""ed"":""https://ksr-ugc.imgix.net/assets/011/398/387/4488ea1b7a56306c87cbffcfeedd9ed3_original.jpg?ixlib=rb-1.1.0&crop=faces&w=352&h=198&fit=crop&v=1463682113&auto=format&frame=1&q=92&s=5dd126641b3160f45361d9a6399d726c"",""med"":""https://ksr-ugc.imgix.net/assets/011/398/387/4488ea1b7a56306c87cbffcfeedd9ed3_original.jpg?ixlib=rb-1.1.0&crop=faces&w=272&h=153&fit=crop&v=1463682113&auto=format&frame=1&q=92&s=11acf4109210b202564d77439e25381b"",""little"":""https://ksr-ugc.imgix.net/assets/011/398/387/4488ea1b7a56306c87cbffcfeedd9ed3_original.jpg?ixlib=rb-1.1.0&crop=faces&w=208&h=117&fit=crop&v=1463682113&auto=format&frame=1&q=92&s=f3fe661f080175984ffc5a3f63e76b29"",""small"":""https://ksr-ugc.imgix.net/assets/011/398/387/4488ea1b7a56306c87cbffcfeedd9ed3_original.jpg?ixlib=rb-1.1.0&crop=faces&w=160&h=90&fit=crop&v=1463682113&auto=format&frame=1&q=92&s=39783454c81688eed1594e247d08f453"",""thumb"":""https://ksr-ugc.imgix.net/assets/011/398/387/4488ea1b7a56306c87cbffcfeedd9ed3_original.jpg?ixlib=rb-1.1.0&crop=faces&w=48&h=27&fit=crop&v=1463682113&auto=format&frame=1&q=92&s=d1c58104c6dc4bc65ed8a45453df42aa"",""1024x576"":""https://ksr-ugc.imgix.net/assets/011/398/387/4488ea1b7a56306c87cbffcfeedd9ed3_original.jpg?ixlib=rb-1.1.0&crop=faces&w=1024&h=576&fit=crop&v=1463682113&auto=format&frame=1&q=92&s=826bee912e24fb2f42fe004e8998d94e"",""1536x864"":""https://ksr-ugc.imgix.net/assets/011/398/387/4488ea1b7a56306c87cbffcfeedd9ed3_original.jpg?ixlib=rb-1.1.0&crop=faces&w=1552&h=873&fit=crop&v=1463682113&auto=format&frame=1&q=92&s=142a35e5bc716668900052d9ee02d4c7""}"
515,"{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""position"":17,""parent_id"":14,""color"":10878931,""urls"":{""web"":{""discover"":""http://www.kickstarter.com/discover/categories/music/rock""}}}",Click here to help KHP finish their first Full Length album,successful,"{""web"":{""project"":""https://www.kickstarter.com/projects/901641891/khp-full-length-album-light-reflective-impaired?ref=discovery_category_newest"",""rewards"":""https://www.kickstarter.com/projects/901641891/khp-full-length-album-light-reflective-impaired/rewards""}}","{""key"":""assets/012/124/992/f0873037712d093703878e5b7c63e305_original.jpg"",""full"":""https://ksr-ugc.imgix.net/assets/012/124/992/f0873037712d093703878e5b7c63e305_original.jpg?ixlib=rb-1.1.0&crop=faces&w=560&h=315&fit=crop&v=1463737943&auto=format&frame=1&q=92&s=84c35cc71f00a76a3876bbd9dcca1b5a"",""ed"":""https://ksr-ugc.imgix.net/assets/012/124/992/f0873037712d093703878e5b7c63e305_original.jpg?ixlib=rb-1.1.0&crop=faces&w=352&h=198&fit=crop&v=1463737943&auto=format&frame=1&q=92&s=4392c83c372eaa6b74b003d0c77d6dfd"",""med"":""https://ksr-ugc.imgix.net/assets/012/124/992/f0873037712d093703878e5b7c63e305_original.jpg?ixlib=rb-1.1.0&crop=faces&w=272&h=153&fit=crop&v=1463737943&auto=format&frame=1&q=92&s=c514b388b5e3743e3ce08b6c2d97b846"",""little"":""https://ksr-ugc.imgix.net/assets/012/124/992/f0873037712d093703878e5b7c63e305_original.jpg?ixlib=rb-1.1.0&crop=faces&w=208&h=117&fit=crop&v=1463737943&auto=format&frame=1&q=92&s=8798ed0430a1db07a0d5d1f6aa911283"",""small"":""https://ksr-ugc.imgix.net/assets/012/124/992/f0873037712d093703878e5b7c63e305_original.jpg?ixlib=rb-1.1

In [50]:
dfnew.name

0       New Final Round Album                                       
1       Princess Pals Enamel Pin Series                             
2       Their Life Through Their Lens-the Amish and Mennonite People
3       WAO: THE ECO EFFECT SHOES                                   
4       Apple Watch Development Course                              
5       Saluki Totem Enamel Pin                                     
6       Quickstarter: Tao Of Maceo                                  
7       #lolnub. Web Theme Park and Story Attraction.               
8       Limited Edition Heroic Fireman Statuette                    
9       PRATTLE                                                     
10      End of an Era — If These Walls Could Talk                   
11      Line In The Sand: Stories from the Northern Gateway         
12      Avalon Nero | Handcrafted Prestige Men's Shoes London Navy  
13      Nakefit                                                     
14      War of Dragons ,Fight to t

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type,combinedfeatures
1051,37,Our first album!,"{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""position"":17,""parent_id"":14,""color"":10878931,""urls"":{""web"":{""discover"":""http://www.kickstarter.com/discover/categories/music/rock""}}}",1315,US,1342680148,"{""id"":1557938215,""name"":""Moon Debris"",""is_registered"":null,""chosen_currency"":null,""avatar"":{""thumb"":""https://ksr-ugc.imgix.net/assets/006/798/349/2da9da468cf40d6e4ab52db83a8c3350_original.JPG?ixlib=rb-1.1.0&w=40&h=40&fit=crop&v=1461412188&auto=format&frame=1&q=92&s=457a45f04f2bf1987876901646db961e"",""small"":""https://ksr-ugc.imgix.net/assets/006/798/349/2da9da468cf40d6e4ab52db83a8c3350_original.JPG?ixlib=rb-1.1.0&w=160&h=160&fit=crop&v=1461412188&auto=format&frame=1&q=92&s=bd9dd8316931f6ae850f527af2738ed1"",""medium"":""https://ksr-ugc.imgix.net/assets/006/798/349/2da9da468cf40d6e4ab52db83a8c3350_original.JPG?ixlib=rb-1.1.0&w=160&h=160&fit=crop&v=1461412188&auto=format&frame=1&q=92&s=bd9dd8316931f6ae850f527af2738ed1""},""urls"":{""web"":{""user"":""https://www.kickstarter.com/profile/1557938215""},""api"":{""user"":""https://api.kickstarter.com/v1/users/1557938215?signature=1552621578.279bb067d77b99dc3c7c834cb8a6016cedca5b5c""}}}",USD,$,True,...,https://www.kickstarter.com/discover/categories/music/rock,True,False,successful,1347860654,1.000000,"{""web"":{""project"":""https://www.kickstarter.com/projects/1557938215/moon-debris-lp?ref=discovery_category_newest"",""rewards"":""https://www.kickstarter.com/projects/1557938215/moon-debris-lp/rewards""}}",1315.00000,international,Rock music/rock first album
515,26,"2 years in the making, this album features songs from acoustic to rock/hip-hop, old to new and everything in between.","{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""position"":17,""parent_id"":14,""color"":10878931,""urls"":{""web"":{""discover"":""http://www.kickstarter.com/discover/categories/music/rock""}}}",1179,US,1430496334,"{""id"":901641891,""name"":""KHP"",""is_registered"":null,""chosen_currency"":null,""avatar"":{""thumb"":""https://ksr-ugc.imgix.net/assets/009/255/708/f0873037712d093703878e5b7c63e305_original.jpg?ixlib=rb-1.1.0&w=40&h=40&fit=crop&v=1461568661&auto=format&frame=1&q=92&s=026ac1a7774f5daa2f964119688ba0a6"",""small"":""https://ksr-ugc.imgix.net/assets/009/255/708/f0873037712d093703878e5b7c63e305_original.jpg?ixlib=rb-1.1.0&w=160&h=160&fit=crop&v=1461568661&auto=format&frame=1&q=92&s=613032195900db0f1b220ade61ee21d2"",""medium"":""https://ksr-ugc.imgix.net/assets/009/255/708/f0873037712d093703878e5b7c63e305_original.jpg?ixlib=rb-1.1.0&w=160&h=160&fit=crop&v=1461568661&auto=format&frame=1&q=92&s=613032195900db0f1b220ade61ee21d2""},""urls"":{""web"":{""user"":""https://www.kickstarter.com/profile/901641891""},""api"":{""user"":""https://api.kickstarter.com/v1/users/901641891?signature=1552621379.5ecc4015d265f63fb58ab94fec93bc4a890ca3b8""}}}",USD,$,True,...,https://www.kickstarter.com/discover/categories/music/rock,True,False,successful,1434517542,1.000000,"{""web"":{""project"":""https://www.kickstarter.com/projects/901641891/khp-full-length-album-light-reflective-impaired?ref=discovery_category_newest"",""rewards"":""https://www.kickstarter.com/projects/901641891/khp-full-length-album-light-reflective-impaired/rewards""}}",1179.00000,international,Rock music/rock album features songs 2 years rock old new making hop hip everything acoustic
331,3,"An album based on influences by a new and old generation of rock and metal, forming a new sound","{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""position"":17,""parent_id"":14,""color"":10878931,""urls"":{""web"":{""discover"":""http://www.kickstarter.com/discover/categories/music/rock""}}}",28,MX,1548729044,"{""id"":1979651714,""name"

In [4]:
import tensorflow as tf
from tensorflow import keras


In [5]:
new_model = tf.keras.models.load_model('testingmedicalmodel.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
new_model